In [4]:
#2D Cavitry Staggered grids (uniform grids)
import numpy as np
from timeit import default_timer as timer
#import cupy as cp

#Global method
def TDMA(a,b,c,d,g,factor,nx,ny,nz,x,y,z):
    if x == 1: #x-direction TDMA
        for i in range(1,nx): #remove of a coefficient
            
            factor[i,:,:,:] = a[i,:,:,:]/b[i-1,:,:,:] #factor,a는 0 index는 버리는 것
            b[i,:,:,:] = b[i,:,:,:] - factor[i,:,:,:]*c[i-1,:,:,:]
            d[i,:,:,:] = d[i,:,:,:] - factor[i,:,:,:]*d[i-1,:,:,:]
            
        g[nx-1,:,:,:] = d[nx-1,:,:,:]/b[nx-1,:,:,:]
        
        for i in range(nx-2,(-1),-1):
            g[i,:,:,:] = (d[i,:,:,:]-c[i,:,:,:]*g[i+1,:,:,:])/b[i,:,:,:]
            
    elif y == 1: #y-direction TDMA
        for j in range(1,ny): #remove of a coefficient
            
            factor[:,j,:,:] = a[:,j,:,:]/b[:,j-1,:,:] 
            b[:,j,:,:] = b[:,j,:,:] - factor[:,j,:,:]*c[:,j-1,:,:]
            d[:,j,:,:] = d[:,j,:,:] - factor[:,j,:,:]*d[:,j-1,:,:]
            
        g[:,ny-1,:,:] = d[:,ny-1,:,:]/b[:,ny-1,:,:]
        
        for j in range(ny-2,(-1),-1):
            g[:,j,:,:] = (d[:,j,:,:]-c[:,j,:,:]*g[:,j+1,:,:])/b[:,j,:,:]
    
    elif z == 1: #y-direction TDMA
        for k in range(1,nz): #remove of a coefficient
            
            factor[:,:,k,:] = a[:,:,k,:]/b[:,:,k-1,:] 
            b[:,:,k,:] = b[:,:,k,:] - factor[:,:,k,:]*c[:,:,k-1,:]
            d[:,:,k,:] = d[:,:,k,:] - factor[:,:,k,:]*d[:,:,k-1,:]
            
        g[:,:,nz-1,:] = d[:,:,nz-1,:]/b[:,:,nz-1,:]
        
        for k in range(nz-2,(-1),-1):
            g[:,:,k,:] = (d[:,:,k,:]-c[:,:,k,:]*g[:,:,k+1,:])/b[:,:,k,:]
    
    return g

def Poisson(p_n,p_o,u_mid,v_mid,dt,nx,ny,dx,dy,epsil_p):
    
    steps = int(0)
    
    while(1):
        
        sum_error = np.zeros([1],dtype = np.float64)
        
        if steps ==0:
            p_o[0,1:ny,:,:] = p_o[1,1:ny,:,:]; p_o[nx,1:ny,:,:] = p_o[nx-1,1:ny,:,:]
            p_o[1:nx,0,:,:] = p_o[1:nx,1,:,:]; p_o[1:nx,ny,:,:] = p_o[1:nx,ny-1,:,:]  
        
        p_n[1:nx,1:ny,:,:] = (p_o[2:nx+1,1:ny,:,:] + p_o[0:nx-1,1:ny,:,:]+p_o[1:nx,2:ny+1,:,:] + p_o[1:nx,0:ny-1,:,:] - dx**2/dt*\
                             ((u_mid[2:nx+1,1:ny,:]-u_mid[1:nx,1:ny,:])/dx +(v_mid[1:nx,2:ny+1,:]-v_mid[1:nx,1:ny,:])/dy))/4.0
        
        #Neumman B.C
        p_n[0,1:ny,:,:] = p_n[1,1:ny,:,:]; p_n[nx,1:ny,:,:] = p_n[nx-1,1:ny,:,:]
        p_n[1:nx,0,:,:] = p_n[1:nx,1,:,:]; p_n[1:nx,ny,:,:] = p_n[1:nx,ny-1,:,:]
        
        sum_error[:] = np.sum(abs(p_n[1:nx,1:ny] - p_o[1:nx,1:ny]))/((nx-1)*(ny-1))
        
        p_o[:]= p_n[:]
        
        if (sum_error<epsil_p):
            break
            
        steps = steps+1
        
    return p_o, steps

def main():
    
    #grids shape
    nx = 129; ny = 129
    
    #---------------------- Define variables ---------------------#
    #Dfine 2-D velocity
    u_o = np.zeros([nx+1,ny+1,1,1], dtype = np.float64);v_o = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    u_n = np.zeros([nx+1,ny+1,1,1], dtype = np.float64);v_n = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    u_c = np.zeros([nx,ny,1,2], dtype = np.float64)
    stream = np.zeros([nx,ny,1,1], dtype = np.float64)
    vorticity = np.zeros([nx-1,ny-1,1,1],dtype = np.float64)
    p_o = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    p_n = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    
    #Coefficient
    #x-direction coefficient
    a_x = np.zeros([nx+1,1,1,1],dtype = np.float64); b_x = np.zeros([nx+1,1,1,1],dtype = np.float64)
    c_x = np.zeros([nx+1,1,1,1],dtype = np.float64); d_u = np.zeros([nx-2,ny-1,1,1],dtype = np.float64)
    factor_x = np.zeros([nx+1,1,1,1],dtype = np.float64)

    #y-direction coefficient
    a_y = np.zeros([1,ny+1,1,1],dtype = np.float64); b_y = np.zeros([1,ny+1,1,1],dtype = np.float64)
    c_y = np.zeros([1,ny+1,1,1],dtype = np.float64); d_v = np.zeros([nx-1,ny-2,1,1],dtype = np.float64)
    factor_y = np.zeros([1,ny+1,1,1],dtype = np.float64)
    
    #intermediate factor for TDMA
    g = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    h = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    u_mid = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    v_mid = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    
    #gradient of velocity
    dudx = np.zeros([nx+1,ny+1,1,1], dtype = np.float64); dvdx = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    dudy = np.zeros([nx+1,ny+1,1,1], dtype = np.float64); dvdy = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    dudx_2 = np.zeros([nx+1,ny+1,1,1], dtype = np.float64); dvdx_2 = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    dudy_2 = np.zeros([nx+1,ny+1,1,1], dtype = np.float64); dvdy_2 = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    
    #Non-linear
    H_uo = np.zeros([nx+1,ny+1,1,1], dtype = np.float64); H_vo = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    H_up = np.zeros([nx+1,ny+1,1,1], dtype = np.float64); H_vp = np.zeros([nx+1,ny+1,1,1], dtype = np.float64)
    #-------------------------------------------------------------#
    
    # Simulation parameter
    Re = 3000
    dt = 0.0025
    t_real =  int(0)
    epsil_p = 10**(-8)
    epsil_u = 10**(-8)
    
    #simulation factor
    x_len = 1.0; y_len = 1.0
    
    x = np.zeros([nx,1,1,1], dtype = np.float64)
    y = np.zeros([1,ny,1,1], dtype = np.float64)
    
    dx = x_len/(nx-1); dy = y_len/(ny-1) #delta panel 칸의 갯수로 나눠줘야함
    x0 = 0.0; y0 = 0.0
    xf = 1.0; yf = 1.0
    
    alpha = 1.0
    video_scene=0
    
    #uniform grids
    x[:,0,0,0] = np.linspace(x0,xf,nx)
    y[0,:,0,0] = np.linspace(y0,yf,ny)
    
    mu = dt/(2*Re*dx**2)
    
    start = timer()
    
    while(1):
        
        #update non-linear term
        H_up[2:nx,1:ny] = H_uo[2:nx,1:ny]
        H_vp[1:nx,2:ny] = H_vo[1:nx,2:ny]
        
        #Boundary condition
        u_o[1:nx,ny,:,:] = (2 - u_o[1:nx,ny-1,:,:]); u_o[1:nx,0,:,:] = (2-u_o[1:nx,1,:,:])
        v_o[nx,1:ny,:,:] = (2- v_o[nx-1,1:ny,:,:]); v_o[0,1:ny,:,:] = (2- v_o[1,1:ny,:,:])
    
        dudx[2:nx,1:ny,:,:] = 0.5*((u_o[3:nx+1,1:ny,:,:] - u_o[2:nx,1:ny,:,:])/dx + (u_o[2:nx,1:ny,:,:] - u_o[1:nx-1,1:ny,:,:])/dx)
        dudy[1:nx,1:ny,:,:] = 0.5*((u_o[1:nx,2:ny+1,:,:] - u_o[1:nx,1:ny,:,:])/dy + (u_o[1:nx,1:ny,:,:] - u_o[1:nx,0:ny-1,:,:])/dy)
        #dudx 범위는 2:nx, 1:ny로 정해진다
        dvdx[1:nx,1:ny,:,:] = 0.5*((v_o[2:nx+1,1:ny,:,:] - v_o[1:nx,1:ny,:,:])/dx + (v_o[1:nx,1:ny,:,:] - v_o[0:nx-1,1:ny,:,:])/dx)
        dvdy[1:nx,2:ny,:,:] = 0.5*((v_o[1:nx,3:ny+1,:,:] - v_o[1:nx,2:ny,:,:])/dy + (v_o[1:nx,2:ny,:,:] - v_o[1:nx,1:ny-1,:,:])/dy)
        
        H_uo[2:nx,1:ny] = u_o[2:nx,1:ny]*dudx[2:nx,1:ny]+0.5*(0.5*(v_o[1:nx-1,1:ny] + v_o[1:nx-1,2:ny+1])+0.5*(v_o[2:nx,1:ny] + v_o[2:nx,2:ny+1]))*dudy[2:nx,1:ny]
        H_vo[1:nx,2:ny] = 0.5*(0.5*(u_o[1:nx,1:ny-1]+u_o[1:nx,2:ny]) + 0.5*(u_o[2:nx+1,1:ny-1]+u_o[2:nx+1,2:ny]))*dvdx[1:nx,2:ny] + v_o[1:nx,2:ny]*dvdy[1:nx,2:ny]
        
        dudx_2[2:nx,1:ny] = ((u_o[3:nx+1,1:ny] - u_o[2:nx,1:ny])/dx - (u_o[2:nx,1:ny] - u_o[1:nx-1,1:ny])/dx)/dx
        dudy_2[1:nx,1:ny] = ((u_o[1:nx,2:ny+1] - u_o[1:nx,1:ny])/dy - (u_o[1:nx,1:ny] - u_o[1:nx,0:ny-1])/dy)/dy
        dvdx_2[1:nx,1:ny] = ((v_o[2:nx+1,1:ny] - v_o[1:nx,1:ny])/dx - (v_o[1:nx,1:ny] - v_o[0:nx-1,1:ny])/dx)/dx
        dvdy_2[1:nx,2:ny] = ((v_o[1:nx,3:ny+1] - v_o[1:nx,2:ny])/dy - (v_o[1:nx,2:ny] - v_o[1:nx,1:ny-1])/dy)/dy
        #Debugging 여기에 0.5가 곱해져 있었음
        
        if t_real ==0: H_up[2:nx,1:ny] = H_uo[2:nx,1:ny]; H_vp[1:nx,2:ny] = H_vo[1:nx,2:ny] #only first step
            
        #d coefficient (Boundary condition은 따로 처리해줘야 한다)
        d_u[:]= dt/(Re)*(dudx_2[2:nx,1:ny] + dudy_2[2:nx,1:ny]) - dt/2*(3*H_uo[2:nx,1:ny] - H_up[2:nx,1:ny]) 
        d_v[:]= dt/(Re)*(dvdx_2[1:nx,2:ny] + dvdy_2[1:nx,2:ny]) - dt/2*(3*H_vo[1:nx,2:ny] - H_vp[1:nx,2:ny])
        
        #------------- u TDMA  ---------------#
        #x-direction TDMA for u velocity
        a_x[2:nx] = -mu; b_x[2:nx] = 1+2*mu; c_x[2:nx] = -mu
        
        x_d=1; y_d=0; z_d=0
        g[2:nx,1:ny,:,:] = TDMA(a_x[2:nx],b_x[2:nx],c_x[2:nx],d_u,g[2:nx,1:ny],factor_x[2:nx],nx-2,ny-1,1,x_d,y_d,z_d)

        #y-direction TDMA for u velocity
        a_y[:,1:ny] = -mu; b_y[:,1:ny] = 1+2*mu; c_y[:,1:ny] = -mu
        b_y[:,1] = (1+3*mu); b_y[:,ny-1] = (1+3*mu)
        
        x_d=0; y_d=1; z_d=0
        u_mid[2:nx,1:ny,:,:] = TDMA(a_y[:,1:ny],b_y[:,1:ny],c_y[:,1:ny],g[2:nx,1:ny],h[2:nx,1:ny],factor_y[:,1:ny],nx-2,ny-1,1,x_d,y_d,z_d)
        
        #------------ v TDMA -----------------#
        #x-direction TDMA for v velocity
        a_x[1:nx] = -mu; b_x[1:nx] = 1+2*mu; c_x[1:nx] = -mu
        b_x[1,:] = (1+3*mu); b_x[nx-1,:] = (1+3*mu)
        
        x_d=1; y_d=0; z_d=0
        g[1:nx,2:ny,:,:] = TDMA(a_x[1:nx],b_x[1:nx],c_x[1:nx],d_v,g[1:nx,2:ny],factor_x[1:nx],nx-1,ny-2,1,x_d,y_d,z_d)

        #y-direction TDMA for v velocity
        a_y[:,2:ny] = -mu; b_y[:,2:ny] = 1+2*mu; c_y[:,2:ny] = -mu
        
        x_d=0; y_d=1; z_d=0
        v_mid[1:nx,2:ny,:,:] = TDMA(a_y[:,2:ny],b_y[:,2:ny],c_y[:,2:ny],g[1:nx,2:ny],h[1:nx,2:ny],factor_y[:,2:ny],nx-1,ny-2,1,x_d,y_d,z_d)
        
        
        #update intermeidate velocity
        u_mid[2:nx,1:ny] = u_o[2:nx,1:ny] + u_mid[2:nx,1:ny]
        v_mid[1:nx,2:ny] = v_o[1:nx,2:ny] + v_mid[1:nx,2:ny]
        
        
        #-------------- Poisson equation ----------------#
        p_o, steps = Poisson(p_n,p_o,u_mid,v_mid,dt,nx,ny,dx,dy,epsil_p)
        #Poisson equation에서 update된 p_o를 받아와야 하는데 p_n을 받아와서 터진적이 있음
        
        #Correction pressure term
        u_n[2:nx,1:ny,:,:] = u_mid[2:nx,1:ny,:,:] - dt*(p_o[2:nx,1:ny] - p_o[1:nx-1,1:ny])/dx #u_update
        v_n[1:nx,2:ny,:,:] = v_mid[1:nx,2:ny,:,:] - dt*(p_o[1:nx,2:ny] - p_o[1:nx,1:ny-1])/dy #v_update
        
        print("Residual")
        print(np.sum(abs(u_n[1:nx,1:ny] - u_o[1:nx,1:ny]) + abs(v_n[1:nx,1:ny] - v_o[1:nx,1:ny])/((nx-1)*(ny-1))))
        
        Residual = (np.sum(abs(u_n[1:nx,1:ny] - u_o[1:nx,1:ny]) + abs(v_n[1:nx,1:ny] - v_o[1:nx,1:ny])/((nx-1)*(ny-1))))
        
        if (Residual < epsil_u):
            break
        

        u_o[:] = u_n[:]; v_o[:] = v_n[:]
        
        
        #print contour stream line & velocity field
        '''
        if t_real% 20==0:
            video_plt = open('2D u_final_video.plt', 'a')

            if t_real==0: video_plt.write('VARIABLES = "x","y","u","v"\n')
            video_plt.write('Zone T="HIT%d"\n'%0)
            video_plt.write('I=%d J=%d\n' %(nx-1, ny-1))
    
            for j in range(1, ny):
                for i in range(1, nx):
                    video_plt.write('%.5f %.5f %.5f %.5f\n' %(i,j, u_o[i,j,0,0], v_o[i,j,0,0]))
    
            video_plt.close()
        '''

            #Boundary condition
            #u_n[1:nx,ny,:,:] = (2 - u_n[1:nx,ny-1,:,:]); u_n[1:nx,0,:,:] = -u_n[1:nx,1,:,:]
            #v_n[nx,1:ny,:,:] = - v_n[nx-1,1:ny,:,:]; v_n[0,1:ny,:,:] = - v_n[1,1:ny,:,:]
            
        if t_real%50==0:
            
            #Staggered grids to Collocated grids
            u_c[0:nx,0:ny,:,0:1] = (u_n[1:nx+1,0:ny] + u_n[1:nx+1,1:ny+1])/2
            u_c[0:nx,0:ny,:,1:2] = (v_n[0:nx,1:ny+1] + v_n[1:nx+1,1:ny+1])/2   
            
            #Get vorticity [d(v_y)/dx - d(v_x)/dy]
        
            vorticity[:] = ((u_c[1:nx,1:ny,:,1:2]-u_c[0:nx-1,1:ny,:,1:2])/dx + (u_c[1:nx,0:ny-1,:,1:2]-u_c[0:nx-1,0:ny-1,:,1:2])/dx)/2-\
            ((u_c[0:nx-1,1:ny,:,0:1] - u_c[0:nx-1,0:ny-1,:,0:1])/dy + (u_c[1:nx,1:ny,:,0:1] - u_c[1:nx,0:ny-1,:,0:1])/dy)/2
            
            '''
            for i in range(nx):
        
                sum_stream = np.zeros([1], dtype = np.float64)
        
                for j in range(ny):
                    sum_stream[:] = sum_stream[:] + u_c[i,j,:,0]*dy
                    stream[i,j] = sum_stream[:]
                    
            stream_video = open('2D stream-line.plt', 'a')

            if t_real==0: stream_video.write('VARIABLES = "x","y","stream"\n')
            stream_video.write('Zone T="HIT%d"\n'%0)
            stream_video.write('I=%d J=%d\n' %(nx, ny))
    
            for j in range(ny):
                for i in range(nx):
                    stream_video.write('%.5f %.5f %.5f\n' %(i,j, stream[i,j,0,0]))
                    
            stream_video.close()     
            '''
            
            vorticity_video = open('2D vorticity.plt', 'a')
            
            if t_real==0: vorticity_video.write('VARIABLES = "x","y","vorticity"\n')
            vorticity_video.write('Zone T="HIT%d"\n'%0)
            vorticity_video.write('I=%d J=%d\n' %(nx-1, ny-1))
    
            for j in range(ny-1):
                for i in range(nx-1):
                    vorticity_video.write('%.5f %.5f %.5f\n' %(i,j, vorticity[i,j,0,0]))
                    
            vorticity_video.close()  
            
            video_scene += 1 
            
            if video_scene == 410: break
            print(video_scene)
            
        t_real += 1               
            
'''        
        
     
    
    duration = timer() - start 
        
    #Boundary condition
    u_n[1:nx,ny,:,:] = (2 - u_n[1:nx,ny-1,:,:]); u_n[1:nx,0,:,:] = -u_n[1:nx,1,:,:]
    v_n[nx,1:ny,:,:] = - v_n[nx-1,1:ny,:,:]; v_n[0,1:ny,:,:] = - v_n[1,1:ny,:,:]
    
    #Staggered grids to Collocated grids
    u_c[0:nx,0:ny,:,0:1] = (u_n[1:nx+1,0:ny] + u_n[1:nx+1,1:ny+1])/2
    u_c[0:nx,0:ny,:,1:2] = (v_n[0:nx,1:ny+1] + v_n[1:nx+1,1:ny+1])/2    
    
    #stream line computation
    for i in range(nx):
        
        sum_stream = np.zeros([1], dtype = np.float64)
        
        for j in range(ny):
            sum_stream[:] = sum_stream[:] + u_c[i,j,:,0]*dy
            stream[i,j] = sum_stream[:]
            
    
    #print contour stream line & velocity field
    contour = open('2D u_final.plt', 'w')

    contour.write('VARIABLES = "x","y","u","v"\n')
    contour.write('Zone T="HIT%d"\n'%0)
    contour.write('I=%d J=%d\n' %(nx-1, ny-1))
    
    for j in range(1, ny):
        for i in range(1, nx):
            contour.write('%.5f %.5f %.5f %.5f\n' %(i,j, u_o[i,j,0,0], v_o[i,j,0,0]))
    
    contour.close()
    
    print("end")
    print(duration)
    
    #print stream line & Collocated grids
    
    contour = open('2D stream-line.plt', 'w')

    contour.write('VARIABLES = "x","y","stream"\n')
    contour.write('Zone T="HIT%d"\n'%0)
    contour.write('I=%d J=%d\n' %(nx, ny))
    
    for j in range(ny):
        for i in range(nx):
            contour.write('%.5f %.5f %.5f\n' %(i,j, stream[i,j,0,0]))
    
    contour.close()    
                
#vorticity (x-direction) d(v_z)/dy - d(v_y)/dz
vorticity_gpu[:,:,:,0] = (up_storage_gpu[1:nyp,1:nzp,1:nxp,2] - up_storage_gpu[0:nyp-1,1:nzp,1:nxp,2])/dy_gpu[0:nyp-1, :, :,0] \
                                        - (up_storage_gpu[1:nyp,1:nzp,1:nxp,1] - up_storage_gpu[1:nyp,0:nzp-1,1:nxp,1])/dz

#vorticity (y-direction) d(v_x)/dz - d(v_z)/dx
vorticity_gpu[:,:,:,1] = (up_storage_gpu[1:nyp,1:nzp,1:nxp,0] - up_storage_gpu[1:nyp, 0:nzp-1,1:nxp,0])/dz \
                                - (up_storage_gpu[1:nyp,1:nzp,1:nxp,2] - up_storage_gpu[1:nyp,1:nzp,0:nxp-1,2])/dx

#vorticity (z-direction) d(v_y)/dx - d(v_x)/dy
vorticity_gpu[:,:,:,2] = (up_storage_gpu[1:nyp,1:nzp,1:nxp,1] - up_storage_gpu[1:nyp,1:nzp,0:nxp-1,1])/dx \
                    - (up_storage_gpu[1:nyp,1:nzp,1:nxp,0] - up_storage_gpu[0:nyp-1,1:nzp,1:nxp,0])/dy_gpu[0:nyp-1, :, :,0]        
        
'''    

            

        
if __name__ == "__main__":
        
    main()
        
    print("All process is finished!")


Residual
0.7838744495466541
1
Residual
0.7884744597330767
Residual
0.7872916440386666
Residual
0.7860529265443175
Residual
0.7847589513208708
Residual
0.783407559937574
Residual
0.7819786381007454
Residual
0.7801402427716022
Residual
0.7761537968685612
Residual
0.7728189279616331
Residual
0.7712993142555042
Residual
0.7697985118623525
Residual
0.7679869640827879
Residual
0.7653776863585736
Residual
0.7630604921864231
Residual
0.7613253261747923
Residual
0.7601816515060559
Residual
0.759229230903129
Residual
0.7580922721785428
Residual
0.756506787314392
Residual
0.7547803466744094
Residual
0.7528646089770877
Residual
0.7509842575912962
Residual
0.7492155215944201
Residual
0.7475525970537593
Residual
0.746428513380885
Residual
0.74527077920586
Residual
0.7427070119659358
Residual
0.7415657730436925
Residual
0.7404314290002283
Residual
0.739223523261997
Residual
0.7355908007615422
Residual
0.7344962818589152
Residual
0.7334022777234342
Residual
0.7308317305602036
Residual
0.72980003831296

Residual
0.43679941428624436
Residual
0.4355069729901159
Residual
0.43463534618101063
Residual
0.4341296035549641
Residual
0.43374035166007785
Residual
0.4332120471844075
Residual
0.4327888026308163
Residual
0.43163843879144725
7
Residual
0.4311725415936743
Residual
0.43062324994156065
Residual
0.43000472319597277
Residual
0.42797647288172025
Residual
0.42728624512905433
Residual
0.42677907216170957
Residual
0.42498765203985606
Residual
0.42463633040243254
Residual
0.4242825991249946
Residual
0.4234858603192264
Residual
0.42275544294330586
Residual
0.42220741037132425
Residual
0.4217572137184855
Residual
0.4212586404340371
Residual
0.4206628757426873
Residual
0.42038673744032407
Residual
0.4193624877552744
Residual
0.4190408989178759
Residual
0.41779755635918925
Residual
0.4168745644083279
Residual
0.4166069379047969
Residual
0.41630457667109527
Residual
0.41599258089889035
Residual
0.41563822966812947
Residual
0.41449269421504015
Residual
0.4132629370144302
Residual
0.4128809374390738

Residual
0.3600758154932149
Residual
0.35975705820701287
Residual
0.3597753032744714
Residual
0.3601580324951875
Residual
0.3607220252599607
Residual
0.3612725003639435
Residual
0.36170376502260987
Residual
0.36191946209151976
Residual
0.36219291241602897
Residual
0.3624333937942956
Residual
0.3627496645129509
Residual
0.36220338076630776
Residual
0.3619126923786981
Residual
0.3622605927298451
Residual
0.36283536373887415
Residual
0.3633722660878138
Residual
0.3638232413254532
Residual
0.36409584036506554
Residual
0.3644004248625398
13
Residual
0.36466339972857287
Residual
0.36394030608871836
Residual
0.36376997158589497
Residual
0.36408326773968436
Residual
0.3645375555492105
Residual
0.36484441006743284
Residual
0.3648825828802976
Residual
0.3647628578447405
Residual
0.36488863804781174
Residual
0.3651540424751637
Residual
0.3654223195252966
Residual
0.36490177451311234
Residual
0.3649296558037121
Residual
0.36509105064057773
Residual
0.3652429112010019
Residual
0.365379685506283
Res

Residual
0.47920984831890334
Residual
0.480504277602662
Residual
0.4812662650605506
Residual
0.482107793408585
Residual
0.4829618863822583
Residual
0.4841034267087176
Residual
0.48521185227149843
Residual
0.4863537590918091
Residual
0.4875746182627226
Residual
0.48901129434429325
Residual
0.4898606172894098
Residual
0.4905856059506186
Residual
0.49168422635733783
Residual
0.49302724450105495
Residual
0.49404936859027215
Residual
0.49504351064006213
Residual
0.4959537018669137
Residual
0.4968426740454006
Residual
0.4977070601440366
Residual
0.4983881329028053
Residual
0.49899911765338867
Residual
0.49956317636780895
Residual
0.5002679209612152
Residual
0.5010971512713549
Residual
0.5020077736994243
Residual
0.5029248590870805
Residual
0.5038081287298223
Residual
0.504827444997012
Residual
0.5054988991842333
19
Residual
0.5061430537058194
Residual
0.5072060557764895
Residual
0.5082962370036874
Residual
0.5095092973042512
Residual
0.5107533510722326
Residual
0.5119744335087187
Residual
0.

Residual
0.7816668985369171
Residual
0.7825808426299019
Residual
0.7836768160681046
Residual
0.7850741412844301
Residual
0.7869608364021226
Residual
0.788733035185047
Residual
0.7905978205112234
Residual
0.792362556133046
Residual
0.7942510448145782
Residual
0.7959968877976853
Residual
0.7976656042636652
Residual
0.7994135163570036
Residual
0.8008621429773286
Residual
0.8024072650078855
Residual
0.8037259933440143
Residual
0.805124809416718
Residual
0.8062628732200907
Residual
0.8072547010042261
Residual
0.8083048011082419
Residual
0.809405257278477
Residual
0.8105773889591285
Residual
0.8118140382135328
Residual
0.8129647156456001
Residual
0.8142015459163781
Residual
0.8158658904801066
Residual
0.8171538280282099
Residual
0.8184207033685382
Residual
0.8198779766378534
Residual
0.8212117767957456
Residual
0.8223939348330468
Residual
0.8235954220734737
Residual
0.8253299781193523
Residual
0.826791417476228
Residual
0.8282979670228341
Residual
0.8299812603285708
Residual
0.83136852005909

...

Residual
11.407617366177005
Residual
11.419158523975021
Residual
11.43090559540438
Residual
11.438763845091653
Residual
11.449386947430845
Residual
11.464198651057353
Residual
11.480384355501599
Residual
11.489315496686652
Residual
11.49640833604877
Residual
11.516246774346598
Residual
11.528431820356627
Residual
11.535407420160105
Residual
11.541520437110286
Residual
11.556569968228473
Residual
11.568072970513699
Residual
11.5844602953704
Residual
11.597624839836822
Residual
11.606152165877278
Residual
11.602914828514427
Residual
11.606602788597197
Residual
11.617104870435064
Residual
11.628492059342552
Residual
11.64238832538295
Residual
11.65862525284569
Residual
11.673274744140542
Residual
11.679546968053753
Residual
11.679308797572922
Residual
11.67766129730258
Residual
11.680437736740991
Residual
11.693623289325377
Residual
11.708062619560158
Residual
11.720465464789278
Residual
11.727782634088777
Residual
11.734121918990668
Residual
11.733625863814062
Residual
11.723198974443044

Residual
10.387453393098504
Residual
10.374967036283746
Residual
10.35478602755618
Residual
10.34004564696397
Residual
10.331542233671026
Residual
10.323576426943852
Residual
10.321130914647396
Residual
10.321315128968681
Residual
10.315495518319757
Residual
10.308278966055557
Residual
10.29875773910095
Residual
10.281182063571952
Residual
10.267460975311916
Residual
10.257837378839984
Residual
10.24827259643623
Residual
10.239595956898853
Residual
10.230647441406148
Residual
10.226637088939363
Residual
10.222901855473115
Residual
10.215836782778762
Residual
10.207654517567935
Residual
10.197431139427206
Residual
10.187217007211082
Residual
10.174806039958597
Residual
10.163548138763568
Residual
10.157412115997086
Residual
10.153208915175407
Residual
10.142640791931582
Residual
10.124406667228167
Residual
10.107785998393052
Residual
10.10119124005934
Residual
10.094286020451124
Residual
10.088539344100006
Residual
10.082278762434068
Residual
10.076296227399727
Residual
10.0658800399345

Residual
11.702368936902692
Residual
11.718790734199004
Residual
11.73393898973929
393
Residual
11.739674871613722
Residual
11.74193594819376
Residual
11.75298818726991
Residual
11.767919920062038
Residual
11.780663652016282
Residual
11.784066077491431
Residual
11.792955839921254
Residual
11.80341559570898
Residual
11.822917026928788
Residual
11.843465560894225
Residual
11.864792878180129
Residual
11.885368142648279
Residual
11.90223705657013
Residual
11.91127893999383
Residual
11.920702347400209
Residual
11.930377273911951
Residual
11.93966229743945
Residual
11.948088668369653
Residual
11.967613866951597
Residual
11.983704921551432
Residual
11.996563517459355
Residual
12.005637621879629
Residual
12.014851337322412
Residual
12.024044373131158
Residual
12.038276912546596
Residual
12.05141873610699
Residual
12.067490743758013
Residual
12.080757089022693
Residual
12.099011595552923
Residual
12.11444876751937
Residual
12.124402335262982
Residual
12.125165512955178
Residual
12.1257343670088

Residual
13.433018957113713
Residual
13.453416791956386
Residual
13.472739565646439
Residual
13.496482708575233
Residual
13.512799759095241
Residual
13.5249251655498
Residual
13.538746266831296
Residual
13.559010852536028
Residual
13.58803945115233
399
Residual
13.611817306138596
Residual
13.627034562332653
Residual
13.640067750167093
Residual
13.66053610601901
Residual
13.684698210755515
Residual
13.711843569397832
Residual
13.730109869208071
Residual
13.743389323713528
Residual
13.753160338568572
Residual
13.772770352078528
Residual
13.792194371993702
Residual
13.811406326971927
Residual
13.826253179918627
Residual
13.845385060338339
Residual
13.86422264496556
Residual
13.879585230574754
Residual
13.895055525772886
Residual
13.908565332150983
Residual
13.929807845889442
Residual
13.953326131367803
Residual
13.973319579373655
Residual
13.990301410760214
Residual
14.01228426115255
Residual
14.030437106959521
Residual
14.053362964549967
Residual
14.078007282303483
Residual
14.1029995750

Residual
17.480163865444148
Residual
17.503273686290697
Residual
17.530861458042448
Residual
17.555193829445663
Residual
17.578782487855438
Residual
17.6099243986514
Residual
17.640990621576474
Residual
17.661773376202177
Residual
17.6806644651492
Residual
17.699152672896222
Residual
17.72344117123149
Residual
17.75472477950418
Residual
17.780951921286125
Residual
17.806881512756583
405
Residual
17.83134159326128
Residual
17.853041205109477
Residual
17.878074299398584
Residual
17.898165513585873
Residual
17.92213563273608
Residual
17.946072148090167
Residual
17.970204908969293
Residual
17.999350074392165
Residual
18.024819928367275
Residual
18.046758309752096
Residual
18.07128192177649
Residual
18.097497557838615
Residual
18.12210688532071
Residual
18.147991901015278
Residual
18.17149332972499
Residual
18.188320524713646
Residual
18.2058064906173
Residual
18.228305948230847
Residual
18.25879897418414
Residual
18.289741823710486
Residual
18.311858509757243
Residual
18.327522224393675
Re